# Modelo de Black-Litterman

Black Litterman propone una alternativa para considera $k$ opiniones sobre la rentabilidad de los activos. Partiendo de la estadística Bayesiana consideran que la rentabilidad de los activos es una variable aleatoria inobservable y que únicamente podremos inferir su función de distribución. La inferencia parte de una creencia a priori. Se utiliza la información adicional para inferior la función de distribución a posteriori. En dicho marco la cartera de equilibrio propuesta por el CAPM es la creencia a priori y la información generada son las opiniones. La estadística bayesiana permiten determinar la función de distribución a posteriori.

La rentabilidad de los $N$ activos que conforman el mercado es una variable aleatoria:
$$r\sim N(\mu,\Sigma) \tag{1}$$

La catera de equilibrio viene determinada por $w_eq$. Si  $\delta$ representa la aversión al riesgo, La prima por riesgo en equilibrio será (Black 1989):
$$\Pi=\delta\Sigma w_{eq} \tag{2}$$

Estimamos la rentabilidad implícitas en equilibrio suponiendo que tenemos dos activos Intel (INTC) y Pfizer (PFE) cuya matriz de varianza convarianza es (multiplicados por $10^4$):
$$\Sigma=\left[\begin{array}{crl}  
46.0 & 1.06 \\
1.06 & 5.3
\end{array}\right]$$

En primer lugar utilizaremos como cartera de mercado un índice basado en el nivel de capitalización. Así, laLa capitalización de Intel es aproximadamente 80.000 millones de US$ y la de PEnfizer de 100.000 millones US$. Las ponderaciones de ambos activos que configuran el índice son:

$$ W_{INTC}=80/180=0.44, W_{PFE}=100/180=0.56 $$

Considerar que $\delta=2.5$.

In [46]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

numpy trata un vector columna de manera diferente a un array unidimensional. Para usar vectores columna de manera consistente, la siguiente función `as_colvec` permite trabajar con un numpy o una matriz numpy de una columna (es decir, un vector columna) y devuelve los datos como un vector columna. 

In [41]:
def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)

In [42]:
np.arange(4)

array([0, 1, 2, 3])

In [43]:
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

In [44]:
def implied_returns(delta, sigma, w):
    """
Obtenemos los rendimientos implícitos en equilibrio por ingeniería inversa.
- delta: Coeficiente de Aversión al Riesgo (escalar)
- sigma: Matriz de Varianzas-Covarianzas (N x N) como DataFrame
- w: Ponderaciones de la cartera de mercado (N x 1) como Series
Rentabilidades vector N x 1 como Series

`squeeze()`es una herramienta útil para manejar y simplificar 
las dimensiones de los arrays, pero no modifica los datos ni 
las operaciones matemáticas subyacentes como 
el producto de matrices
       """
    ir = delta * sigma.dot(w).squeeze() 
    ir.name = 'Implied Returns'
    return ir


In [45]:
tickers = ['INTC', 'PFE']
s = pd.DataFrame([[46.0, 1.06], [1.06, 5.33]], index=tickers, columns=tickers) *  10E-4
pi = implied_returns(delta=2.5, sigma=s, w=pd.Series([.44, .56], index=tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

La opinión de los analistas disiente de las rentabilidades de equilibrio estimadas y considera que Intel reducirá su rentabilidad al 2% y Pfizer la incrementará al 4%. Por tanto vamos a construir una nueva cartera, no a partir del modelo de mercado, sino a partir del modelo de Markowitz. Por tanto, optimizaremos el ratio de Sharpe considerando que la rentabilidad libre de riesgo es del 0%. 
$$\begin{align}
\mu_e- R_f &=\Sigma z \\
z &=\Sigma^{-1} (\mu_e- R_f) \\
w_{SR_i} &=\frac{z_i}{\sum_{i=1}^{N}z_i} 

\end{align}$$

In [50]:
mean_returns_minus_rf=([.02, .04])
z = np.dot(np.linalg.inv(s), mean_returns_minus_rf)
z

array([0.26305395, 7.45237576])

In [51]:
w_sr = np.divide(z, np.sum(z))
w_sr

array([0.03409453, 0.96590547])

Creamos una función que nos permita realizar las estimaciones anteriores

In [48]:
# se define una función que estima la inversa de la matriz implícita en un dataframe.
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Estimamos la cartera M de acuerdo al modelo de Markowitz (La tangente/Maximzar el Ratio Sharpe) 
    Mu: es el vector de la prima por riesgo, es un seire
    Sigma: es una matriz N x N es un DataFrame
        """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) 
    return w


In [49]:
mu_exp = pd.Series([.02, .04],index=tickers) # INTC and PFE
np.round(w_msr(s, mu_exp)*100, 2)

INTC     3.41
PFE     96.59
dtype: float64

La optimización ingenua de Markowitz propone una cartera con una ponderación poco realista de más del 96% en Pfizer y menos del 4% en Intel. Esto es completamente impracticable y ningún inversor razonable haría apuestas tan dramáticas.

## La propuesta del modelo de Black Litterman

Desde la perspectiva Bayesiana la función a priori de la rentabilidad esperada, está centrada en el valor de equilibrio, es decir, se comporta como un normal de media $\Pi$
$$\mu=\Pi+\varepsilon^{(e)} \tag{3}$$

donde $\varepsilon^{(e)}$ es un vector aleatorio que se distribuye como una normal de media cero y con una matriz de covarianza $\tau\Sigma$, donde $\tau$ es un indicador de la incertidumbre del CAPM a priori. Dicho parámetro es fuente de confusión, algunos los igualan a 1 o calibran el modelo con $\tau$. Black y Litterman proponen un número bajo, en este sentido una técnica habitual es $\tau=1/T$ donde T es el número de periodos utilizados.

Se disponen de $K$ opiniones que se comportan como una normal de media $q$ y desviación típica $\omega$. Dichas opiniones se representan con dos vectores $P$ y $Q$:
$$P'=(p_1, p_2, ...p_k)\tag{4}$$
$$Q'=(q_1, q_2, ...q_k)\tag{5}$$

La opinión de los inversores se puede expresar como:

$$P\mu=Q+\varepsilon^{(\nu)} \tag{6}$$

donde $\varepsilon^{(\nu)}$ es una variable aleatoria inobservable normalmente distribuida con media cero y matriz de covarianza $\Omega$. Se supone que las opiniones son únicas e incorrelacionadas con el resto. Ello permite que la matriz de covarianza que refleja la incertidumbre implícita en las opiniones $\Omega$ sea una matriz diagonal que recoja la varianza de las opiniones $(\omega_i)$. Además se asume que $\varepsilon^{(e)}$ son independientes $\varepsilon^{(\nu)}$:

$$\left(\begin{array}{crl}  
\varepsilon^{(e)} \\
\varepsilon^{(\nu)}
\end{array}\right) \sim 
N\left(0,\left[\begin{array}{crl}  
\tau\Sigma & 0 \\
0 & \Omega
\end{array}\right]\right)
\tag{7}$$



In [31]:
# Opinión absoluta 1: la rentabilidad de INTC será del 2%
# Opinión absoluta 2: la rentabilidad de PFE será del 4%
q = pd.Series({'INTC': 0.02, 'PFE': 0.04})

# La Matriz P (Pick) 
p = pd.DataFrame([
    {'INTC': 1, 'PFE': 0},
    {'INTC': 0, 'PFE': 1}
    ])


INTC    0.02
PFE     0.04
dtype: float64

## La especificación de $\Omega$

Posiblemente no siempre resulta sencillo determinar la varianza asociada a las opiniones, sobre todo cuando trabajamos con opiniones cualitativas. Además, $\left[P^T\Omega^{-1} P \right]$ no es invertible y por tanto no se puede utilizar. Afortunadamente disponemos de distintos métodos para especificar $\Omega$:
- Una proporción de la varianza histórica.
- Un intervalo de confianza.
- La varianza de los residuos en un modelo factorial.
- El método de Idzorek para especificar la confianza a lo largo de la dimensión de las ponderaciones.

### Una proporción de la varianza historica

Se asume que la varianza de las opiones son proporcionales a la varianza de la rentabilidad de los activos (He and Litterman. 1999; Meucci, 2006):

$$\begin{aligned}
\omega_{ij}&=p(\tau\Sigma)p^T \forall i=j \tag{12}\\
\omega_{ij}&=0 \\
\Omega&=diag\left(P (\tau\Sigma) P^T \right)
\end{aligned}$$

### Un intervalo de confianza.

El inversor especifica la varianza utilizando un intervalo de confianza en torno a la rentabilidad media. Por ejemplo, se ha estimado una rentabilidad esperada del 3% y se le asigna una probabilidad del 68% de se encuentre en el intervalo (2%,4%), por tanto la desviación típica es del 1%.

###  La varianza de los residuos en un modelo factorial

La predicción de la rentabilidad puede estar basada en un modelo factorial.

$$r=\sum_{i=1}^{n} \beta_i f_i+ \varepsilon \tag{13}$$

donde:
- $r$ es la rentabilidad del activo.
- $\beta_i$ es el coeficiente del factor $i$.
- $f_i$ es la rentabilidad del factor $i$.
- $\varepsilon$ es el error que presenta una distribución normal. 

La varianza de la rentabilidad estimada a través del modelo factorial es:

$$V(r)=BV(F)B^T+V(\varepsilon) \tag{14}$$

donde:
- $B$ es el vector de coeficientes
- $F$ es el vector de rentabilidad de los factores

Beach y Orlov (2006) realizan las predicciones utilizando un modelo factorial GARCH que posteriormente incluye en el modelo de  Black-Litterman. 

### El método de Idzorek para especificar la confianza a lo largo de la dimensión de las ponderaciones

Idzorek (2005) describe un método para especificar la confianza en la vista en términos de un desplazamiento porcentual de los pesos en el intervalo desde el 0% de confianza hasta el 100% de confianza. Veremos el algoritmo de Idzorek en la sección sobre extensiones


Entre las distintas alternativas definimos una función para estimar el riesgo de las opiniones $\Omega$ a partir de la varianza histórica de los activos.

In [55]:
# Se asume que Omega es proporcional a la varianza histórica
def proportional_prior(sigma, tau, p):
    """
    Basado en He & Litterman (1999) 
    Inputs:
    sigma: Matriz de covarianza N x N es un DataFrame
    tau: es un scalar que mide el nivel de confianza en el CAPM
    p: es un datraframe K x N DataFrame y enlaza las opiniones  Q  con la rentabilidad de los activos 
    
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

## Estimación de la rentabilidad esperada considerando las opiniones

Las opiniones se combinan con el CAPM en el marco de la estadística Bayesiana y el resultado es que las rentabilidades esperadas se distribuyen como $N(\bar{\mu},\bar{M}^{-1} )$. Donde la rentabilidad esperada es:

$$\bar{\mu}=\left[(\tau\Sigma)^{-1}+P'\Omega^{-1}P\right]^{-1}\left[(\tau\Sigma)^{-1}\Pi+P'\Omega^{-1}Q\right] \tag{8}$$

y la matriz de covarianza $\bar{M}^{-1}$ es dado por:

$$\bar{M}^{-1}=\left[(\tau\Sigma)^{-1}+P'\Omega^{-1}P\right]^{-1} \tag{9}$$

In [53]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

In [54]:
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC':.44, 'PFE':.56}), sigma_prior=s, p=p, q=q)
# Black Litterman Implied Mu
bl_mu


INTC    0.037622
PFE     0.024111
dtype: float64

## Identificación de las carteras eficientes de acuerdo a Black- Litterman

Tras considerar las opiniones la rentabilidad presenta la siguiente función de distribución:

$$r\sim N(\bar{\mu},\bar{\Sigma} ) \tag{10}$$

donde $\bar{\Sigma}=\Sigma+\bar{M}^{-1}$

Podremos identificar las cartera eficientes optimizando:

$$max \text{   } \omega'\bar{\mu}-\frac{\delta}{2}\omega'\bar{\Sigma}\omega \tag{11}$$

La condición de primera orden: 
$$\bar{\mu}=\delta\bar{\Sigma}\omega^* \tag{12}$$

de donde:

$$\omega^*=\frac{1}{\delta}\bar{\Sigma}^{-1}\bar{\mu} \tag{13}$$

Utilizando la ecuación 8, podemos estimar la cartera de mercado

$$\omega^*=\frac{1}{\delta}\bar{\Sigma}^{-1}\bar{M}^{-1}\left[(\tau\Sigma)^{-1}\Pi+P'\Omega^{-1}\right] \tag{14}$$

Dado que:
$$\bar{\Sigma}^{-1}=(\Sigma+\bar{M}^{-1})^{-1}=\bar{M}-\bar{M}(\bar{M}+\Sigma^{-1})^{-1}\bar{M} \tag{15}$$

el término $\bar{\Sigma}^{-1}\bar{M}^{-1}$ puede ser simplificado como

$$\bar{\Sigma}^{-1}\bar{M}^{-1}=\frac{\tau}{1+\tau}\left(I-P'A^{-1}P\frac{\Sigma}{1+tau}\right) \tag{16}$$

donde $A=\frac{\Omega}{\tau}+P\frac{\Sigma}{(1+\tau}P'$ por tanto la ponderación de la nueva cartera de mercado será:
$$\omega^*=\frac{1}{1+\tau}(\omega_{eq}+P'\times \Lambda )\tag{17}$$

donde $\omega_{eq}=(\delta\Sigma)^{-1}\Pi$ son las ponderaciones de la cartera en equilibrio y:

$$\Lambda=\tau\Omega^{-1}\frac{Q}{\delta}-A^{-1}P\frac{\Sigma}{1+\tau}\omega_{eq}-A^{-1}P\frac{\Sigma}{1+\tau}P'\tau\frac{Q}{\delta} \tag{18}$$

Dado que cada columna de la matriz $P'$ es una opinión, la ecuación 17 que un inversor crea una cartera compuesta por las ponderaciones de una cartera de equilibrio más más una suma ponderada del impacto de las opiniones condicionadas por un factor $\frac{1}{(1+\tau)}$.

Podemos realizar una interpretación intuitiva de las ponderaciones de las ecuación 18:
- $\tau\Omega^{-1}\frac{Q}{\delta}$ cuanto mayor es el nivel de confianza en una opinión $q_k$ mayor es su peso en la determinación de la ponderación.
- $A^{-1}P\frac{\Sigma}{1+\tau}\omega_{eq}$ la correlación entre los activos penaliza la corrección de las ponderaciones de equilibrio
- $A^{-1}P\frac{\Sigma}{1+\tau}P'\tau\frac{Q}{\delta}$ podemos observar que la penalización por las covarianzas es doble.

Para un inversor con un nivel de aversión al riesgo $\hat{\delta}$ las ponderaciones de la cartera vendrán determinadas por
$$\hat{\omega}^*=\frac{\delta}{\hat{\delta}}\omega^*\tag{19}$$

Para un inversor que no desea asumir un riesgo superior a $\sigma$, tendrá que realizar la siguiente optimización:

$$\underset{\omega'\Sigma \omega\leq \sigma^2}{max} \omega' \bar{\mu} \tag{20}$$

La cartera óptima sería:

$$\tilde{\omega}^*=\left(\frac{\sigma \delta}{\sqrt{\bar{\mu}'\Sigma \bar{\mu}}}\right)\omega^*$$

